In [43]:
with open('../../my_key.txt') as f:
    key_list = f.readlines()

key_list = [key.strip() for key in key_list]

keys = {}
for key in key_list:
    key = key.split(':')
    if (key != ['']):
        keys[key[0]] = key[1]

In [44]:
import os

os.environ['OPENAI_API_KEY'] = keys['OPENAI_API_KEY']

# One Call Information Engine
- Complete the information summarization with one call

## tiktoken

In [22]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

In [8]:
encoding.encode("I love cat")

[40, 3021, 8415]

In [9]:
with open('text1.txt', 'r') as f:
    text1_content = f.read()

with open('text2.txt', 'r') as f:
    text2_content = f.read()


num_tokens = len(encoding.encode(text1_content)) + len(encoding.encode(text2_content))

print(num_tokens)

1688


## OpenAI Original Call
- prompt 1765 + completion 226 = 
    - 'gpt-3.5-turbo': 18s+14s+15s /3 = 16s
    - 'text-davinci-003': above 22s

In [48]:
import openai
import time

openai.organization = keys['OPENAI_ORGANIZATION']
openai.api_key = os.getenv('OPENAI_API_KEY')
# openai.Model.list()

### gpt-3.5-turbo
- Without Modification

In [ ]:
sys_prompt = """
Write a concise summary on the content. You may make some comparisons on different source.
The point you say must include the source, 
for example, [url1] or [url2]"""

text = text1_content + '[from url1]' + text2_content + '[from url2]'

usr_prompt = f"""
{text}
"""

start = time.time()
tur_response = openai.ChatCompletion.create(
    model = 'gpt-3.5-turbo',
    messages = [
        {"role": "system", "content": f'{sys_prompt}'},
        {"role": 'user', 'content': f'{usr_prompt}'},
    ],
    temperature = 0,
)
stop = time.time()

In [55]:
print(str(tur_response) + '\n' + str(stop - start) + 's')

{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "The article compares two popular language models, GPT-3 and BERT, and discusses their differences and similarities, exploring their capabilities and the tools that use them. GPT-3 is an autoregressive language model developed by OpenAI, while BERT is a bidirectional transformer model developed by Google AI. The main difference between the two models lies in their architecture and training dataset size, with GPT-3 being better suited for tasks such as summarization or translation, while BERT is more beneficial for sentiment analysis or natural language understanding (NLU). However, both models have proven themselves valuable tools for performing various NLP tasks with varying degrees of accuracy.",
        "role": "assistant"
      }
    }
  ],
  "created": 1685508068,
  "id": "chatcmpl-7M7vwE8NQeN2sDGdZBmt35eFvgzdK",
  "model": "gpt-3.5-turbo-0301",
  "object": "chat.completio

In [56]:
tur_response['choices'][0]['message']['content']

'The article compares two popular language models, GPT-3 and BERT, and discusses their differences and similarities, exploring their capabilities and the tools that use them. GPT-3 is an autoregressive language model developed by OpenAI, while BERT is a bidirectional transformer model developed by Google AI. The main difference between the two models lies in their architecture and training dataset size, with GPT-3 being better suited for tasks such as summarization or translation, while BERT is more beneficial for sentiment analysis or natural language understanding (NLU). However, both models have proven themselves valuable tools for performing various NLP tasks with varying degrees of accuracy.'

### gpt-3.5-turbo
- With Streaming
    - use 'stream = True'
    - automatically call 'await'

In [62]:
sys_prompt = """
Write a concise summary on the content. You may make some comparisons on different source.
The point you say must include the source, 
for example, [url1] or [url2]"""

text = text1_content + '[from url1]' + text2_content + '[from url2]'

usr_prompt = f"""
{text}
"""

tur_response = openai.ChatCompletion.create(
    model = 'gpt-3.5-turbo',
    messages = [
        {"role": "system", "content": f'{sys_prompt}'},
        {"role": 'user', 'content': f'{usr_prompt}'},
    ],
    temperature = 0,
    stream = True
)

for chunk in tur_response:
    try:
        print(chunk['choices'][0]['delta']['content'])
    except:
        pass

The
 article
 compares
 two
 popular
 language
 models
,
 G
PT
-
3
 and
 B
ERT
,
 discussing
 their
 differences
 and
 similarities
,
 exploring
 their
 capabilities
,
 and
 discussing
 some
 of
 the
 tools
 that
 use
 them
.
 G
PT
-
3
 is
 an
 aut
ore
gressive
 language
 model
 developed
 by
 Open
AI
,
 while
 B
ERT
 is
 a
 bid
irectional
 transformer
 model
 developed
 by
 Google
 AI
.
 The
 main
 difference
 between
 the
 two
 models
 lies
 in
 their
 architecture
 and
 training
 dataset
 size
,
 with
 G
PT
-
3
 being
 better
 suited
 for
 tasks
 such
 as
 summar
ization
 or
 translation
,
 while
 B
ERT
 is
 more
 beneficial
 for
 sentiment
 analysis
 or
 natural
 language
 understanding
 (
N
LU
).
 However
,
 both
 models
 have
 proven
 themselves
 valuable
 tools
 for
 performing
 various
 N
LP
 tasks
 with
 varying
 degrees
 of
 accuracy
.


### text-davinci-003

In [50]:
text = text1_content + '[from url1]' + text2_content + '[from url2]'

usr_prompt = f"""
Write a concise summary on the content. You may make some comparisons on different source.
The point you say must include the source, 
for example, [url1] or [url2]
content: {text}
summary:
"""

start = time.time()
da_response = openai.Completion.create(
    engine = 'text-davinci-003',
    prompt = usr_prompt,
    max_tokens = 300
)
stop = time.time()

In [51]:
da_response

<OpenAIObject text_completion id=cmpl-7M6v1z4mPhssrfNtkAWmzPXFAdDk4 at 0x7f85c38ca150> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "GPT-3 and BERT are two well-known language models used in natural language processing (NLP) applications. GPT-3 is an autoregressive model trained on 45TB of data while BERT is bidirectional and trained on 3TB of data. They have significant differences in architecture and data sizes, but both are Transformer-based and can be used for similar tasks such as question answering, summarization, and translation. GPT-3 typically outperforms BERT in tasks that require more data, while BERT is better for sentiment analysis and natural language understanding. GPT-2 models can be used to generate data from prompts, while BERT can be used for zero- and few-shot learning with the PET method. [from url1 and url2]"
    }
  ],
  "created": 1685504167,
  "id": "cmpl-7M6v1z4mPhssrfNtkAWmzPXFAdDk4",
  "m

In [52]:
da_response['choices'][0]['text']

'GPT-3 and BERT are two well-known language models used in natural language processing (NLP) applications. GPT-3 is an autoregressive model trained on 45TB of data while BERT is bidirectional and trained on 3TB of data. They have significant differences in architecture and data sizes, but both are Transformer-based and can be used for similar tasks such as question answering, summarization, and translation. GPT-3 typically outperforms BERT in tasks that require more data, while BERT is better for sentiment analysis and natural language understanding. GPT-2 models can be used to generate data from prompts, while BERT can be used for zero- and few-shot learning with the PET method. [from url1 and url2]'

## Text Processing
- give text_content

### Web Content Crawling

In [35]:
with open('web_link.txt', 'r') as f:
    web_link = f.readlines()

web_link = [web.strip() for web in web_link]

print(web_link)

['https://www.makeuseof.com/gpt-vs-bert/', 'https://aitechtrend.com/gpt-3-vs-bert-which-one-wins-for-nlp-tasks/', 'https://blog.invgate.com/gpt-3-vs-bert', 'https://symbl.ai/blog/gpt-3-versus-bert-a-high-level-comparison/']


#### Using Unstructured

In [68]:
from langchain.document_loaders import UnstructuredURLLoader

url_loader = UnstructuredURLLoader(
    urls = web_link,
    continue_on_failure = True,
    mode = 'single'
)

url_data = url_loader.load()

In [70]:
url_data

[Document(page_content="ITSM\nTicket Volume\nWorkplace\nProduct\nCase Studies\nGuides\n\n\nMeet InvGate\n\n\n\n\n\n\nEnglish\n\n\n\n\n\nEspañol\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nEnglish\n\n\n\n\n\n\n\nEspañol\n\n\n\n\n\n\n\n\n\nCategories\n\n\n\n\nITSM\n\n\n\n\n\nWorkplace\n\n\n\n\n\nProduct\n\n\n\n\n\nCase Study\n\n\n\n\n\n\n\nLanguages\n\n\nEnglish\n \nEspañol\n\nGPT-3 vs. BERT: Comparing the Two Most Popular Language Models\n\nCeleste Mottesi\n            February 9, 2023\n\n3 min read\n\nNatural language processing (NLP) has come a long way over the past few years. With the development of powerful new models such as GPT-3 and BERT, it's now possible to create sophisticated applications that can understand and interact with human language.\n\nHowever, what went viral as a disruptive chatbot with ChatGPT, suddenly became a contest of language models to power AI content. So, we decided to oppose GPT-3 vs. BERT to understand their differences and similarities, explore their ca

#### Using BeautifulSoup
- Maybe Scrapy Later

In [36]:
import requests
from bs4 import BeautifulSoup

def get_text_content_from_link(url):
    # Send a GET request to the URL
    response = requests.get(url)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find the desired elements and extract their text content
        text_content = ''
        for element in soup.find_all('p'):  # Example: Extract text from all <p> tags
            text_content += element.get_text() + '\n'

        return text_content

    else:
        print(f"Failed to retrieve content from {url}. Status code: {response.status_code}")
        return None

In [82]:
text_content = ''
for i in range(len(web_link)):
    try:
        text_content += ''.join(['\n',f'[this text is from url{i}]', '\n', get_text_content_from_link(web_link[i])])
        print(i)
    except:
        pass


0
1
2
Failed to retrieve content from https://symbl.ai/blog/gpt-3-versus-bert-a-high-level-comparison/. Status code: 403


In [83]:
with open('1.txt', 'w') as f:
    f.write(text_content)

# print(text_content)

In [39]:
type(text_content)

str

In [60]:
num_tokens = len(encoding.encode(text_content))
print(num_tokens)

3163


## One Call Information Engine

In [45]:
import openai

openai.organization = keys['OPENAI_ORGANIZATION']
openai.api_key = os.getenv('OPENAI_API_KEY')

In [84]:
sys_prompt = """You are a summary writer."""

prompt1 = """Write a summary on those web text. You may make some comparisons on the text from different sources."""
prompt2 = "Yes, I will write it with sequence number 1,2,3"

usr_prompt = f"""
<Web texts: (The content is from web so there might be some noise, you should ignore them)
{text_content}>

<Each point you conclude must include the source and output in Sequence, for example:
1 "Point a"[url1]\n
1.1 "Point b"[url1][url2]\n
2 "point c"[url3]>\n

Summary: (Write in detail and with reference, remember to include the source)
"""

tur_response = openai.ChatCompletion.create(
    model = 'gpt-3.5-turbo',
    messages = [
        {"role": "system", "content": f'{sys_prompt}'},
        {"role": 'user', "content": prompt1},
        {"role": 'assistant', "content": prompt2},
        {"role": 'user', 'content': f'{usr_prompt}'},
    ],
    # max_tokens = 2000,
    # stream = True
)

#for chunk in tur_response:
#    try:
#        print(chunk['choices'][0]['delta']['content'])
#    except:
#        pass

In [85]:
tur_response

<OpenAIObject chat.completion id=chatcmpl-7MApIcPRVqhgjSFEzP8z7FmCln3zF at 0x7fe28e765190> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "1. Both GPT and BERT are natural language processing (NLP) models that can perform a range of NLP tasks, like language generation, text classification, and question answering.\n1.1 \"GPT-3 (Generative Pre-trained Transformer 3) is an autoregressive language model developed by OpenAI. It is a language model that can generate natural language text, answer questions, compose poetry, and write complete articles. It has 175 billion parameters, making it one of the largest language models ever constructed.\" [url0]\n1.2 \"BERT (Bidirectional Encoder Representations from Transformers) is a pre-training language representation model that fine-tunes NLP applications created by Google in 2018. BERT uses bidirectional flow, which allows it to use context from both directions during processing

In [86]:
with open('result.txt', 'w') as f:
    f.write(tur_response['choices'][0]['message']['content'])
